# Refactoring code using dependency injection with an example

* Refactoring: improve existing code to improve operation without altering functionality
    + benefits: reusability, testability, maintainability

In [2]:
import math
# set-up code to create necessary files and imports

from pathlib import Path
import requests
from PIL import Image, ImageDraw, ImageFont
tmp_dir = Path(".") / "tmp"

if not tmp_dir.is_dir():
    tmp_dir.mkdir()

font_url = "https://raw.githubusercontent.com/googlefonts/opensans/main/fonts/ttf/OpenSans-Regular.ttf"
font_file = tmp_dir / "OpenSans-Regular.ttf"

if not font_file.exists():
    with open(font_file, "wb") as f:
        r = requests.get(font_url)
        f.write(r.content)

def draw_pattern(image: Image, size=(400, 400), depth=11, angle=30, length=80):
    draw = ImageDraw.Draw(image)
    def draw_branch(x, y, angle_deg, length, depth):
        if depth == 0:
            return
        angle_rad = math.radians(angle_deg)
        x2 = x + length * math.cos(angle_rad)
        y2 = y - length * math.sin(angle_rad)
        draw.line((x ,y, x2, y2), fill="black", width=1)
        draw_branch(x2, y2, angle_deg - angle, length, depth - 1)
        draw_branch(x2, y2, angle_deg + angle, length, depth - 1)
    start_x, start_y = size[0] // 2, 10
    draw_branch(start_x, start_y, -90, length, depth)

image_file = tmp_dir / "base_image.jpg"
if not image_file.exists():
    image = Image.new(mode="RGB",  size=(875, 1125), color=(255,255,255))
    draw_pattern(image, size=(875, 1225), length=400, depth=20)

    image.save(image_file)


## Example original code:
* The following is the proof-of-concept code for a project to create a "yearbook" of photos. At the end, the person's name will be added to the corresponding jpeg.
* Making changes to the code is difficult in this code. Adding text to multiple images is not easy. This is fine for one-time-use code or for a POC.

```mermaid
    flowchart LR
        A[open source file]
        B[calculate text position]
        C[declare font]
        D[declare color]
        E[declare text]
        F[draw text on image]
        G[save file to destination]
        A --> B --> C --> D --> E --> F --> G

```


In [ ]:
img = Image.open(Path("tmp/base_image.jpg")).convert("RGB")

img_w, img_h = img.size
top, bottom, left, right = (0, img_h, 0, img_w)
text_w, text_h = (right - left) // 2, (bottom - top) // 2

font = ImageFont.truetype(Path("tmp/OpenSans-Regular.ttf"), 72)
text = "base"
fillcolor = (255, 0, 255)

drawer = ImageDraw.Draw(img)
drawer.multiline_text(xy=(text_w, text_h), text=text, font=font, align="center", fill=fillcolor)

img.save("tmp/test_0.jpg")
img.close()


## Using functions to abstract functionality
* Functions in programming are small chunks of code that perform specific tasks. Functions take inputs (arguments) and gives outputs (return value).
* The simplest way to create a function in the example is to wrap the original code in a function with no arguments and return a null value.
```mermaid
    flowchart LR
        subgraph function
            A[open source file]
            B[calculate text position]
            C[declare font]
            D[declare color]
            E[declare text]
            F[draw text on image]
            G[save file to destination]
            A --> B --> C --> D --> E --> F --> G
        end

```

In [ ]:
def add_text_0() -> None:
    img = Image.open(Path("tmp/base_image.jpg")).convert("RGB") # make source image more flexible
    img_w, img_h = img.size

    top, bottom, left, right = (0, img_h, 0, img_w)
    text_w, text_h = (right - left) // 2 , (bottom - top) // 2

    font = ImageFont.truetype("tmp/OpenSans-Regular.ttf", 72)
    fillcolor = (255, 0, 255)
    text = "text base" # make text more flexible

    drawer = ImageDraw.Draw(img)
    drawer.multiline_text(xy=(text_w, text_h), text=text, font=font, align="center", fill=fillcolor)
    img.save("tmp/test_0.jpg") # make dst image more flexible
    img.close()

add_text_0()


## Dependency injection
* Dependency injection is a design pattern where an object or function receive necessary information from an external source, which allows for code reusability.
* In this example, I started by changing the source file, destination file, and text into parameters for the function. This allows me to add text to multiple files with a simple for loop.

```mermaid
    flowchart LR
        subgraph dependencies
            fs("source")
            fd("destination")
            text["text"]
        end
        subgraph function
            A[open source file]
            B[calculate text position]
            C[declare font]
            D[declare color]
            E[draw text on image]
            F[save file to destination]
            A --> B --> C --> D --> E --> F
        end

        fs --> A
        fd --> F
        text --> E

```

In [ ]:
def add_text_1(src: Path, dst: Path, text: str) -> None:
    img = Image.open(src).convert("RGB")
    img_w, img_h = img.size
    top, bottom, left, right = (0, img_h, 0, img_w)
    text_w, text_h = (right - left) // 2, (bottom - top) // 2 # make the determination of x,y coordinates more flexible.

    font = ImageFont.truetype("tmp/OpenSans-Regular.ttf", 72)
    fillcolor = (0, 0, 255)

    drawer = ImageDraw.Draw(img)
    drawer.multiline_text(xy=(text_w, text_h), text=text, font=font, align="center", fill=fillcolor)

    img.save(dst)
    img.close()

data = [
    (Path("tmp/base_image.jpg"), Path("tmp/test_1a.jpg"), "text di a"),
    (Path("tmp/base_image.jpg"), Path("tmp/test_1b.jpg"), "text di b")
]

for src, dst, text in data:
        add_text_1(src, dst, text)


## More abstraction
* In addition to primitive types, we can also pass objects and functions as dependencies.
* In this example:
    * A "Text" dataclass to better organize the font, color, and text parameter for the function, which allows for more flexibility.
    * A new function also requires a function to be passed to calculate the anchor for the text. A default function is used to make callinig the function easier.
    * Warning: avoid using default values for mutable type. The default values are evaluated once at function instantiation and not re-evaluated at run time.
```mermaid
    flowchart LR
        subgraph dependencies
            fs("source")
            fd("destination")
            text("<b>Text object</b> <br/> font <br/>  text <br/> alignment <br/> color")
            calc("<b>function to calculate position</b>")
        end
        subgraph function
            A[open source file]
            B[calculate text position]
            E[draw text on image]
            F[save file to destination]
            A --> B --> E --> F
        end
        fs --> A
        fd --> F
        text --> E
        calc --> B
```


In [ ]:
from dataclasses import dataclass
from collections.abc import Callable

@dataclass
class Text:
    """
    Use dataclass to combine all relevant variables in a logical unit.
    """
    font: ImageFont.FreeTypeFont
    text: str
    alignment: str
    color: tuple[int, int, int]

def xy_at_center(img: Image) -> tuple[int, int]:
    img_w, img_h = img.size
    top, bottom, left, right = (0, img_h, 0, img_w)
    text_w, text_h = (right - left) // 2, (bottom - top) // 2
    return text_w, text_h

def xy_at_top_left(img: Image) -> tuple[int, int]:
    img_w, img_h = img.size
    top, bottom, left, right = (0, img_h, 0, img_w)
    return left, top

def add_text_2(
        src: Path,
        dst: Path,
        text: Text,
        xy_function: Callable[[Image], (int, int)] = xy_at_center # default argument
) -> None:
    img = Image.open(src).convert("RGB")

    text_w, text_h = xy_function(img)

    drawer = ImageDraw.Draw(img)
    drawer.multiline_text(xy=(text_w, text_h), text=text.text, font=text.font, align=text.alignment, fill=text.color)

    img.save(dst)
    img.close()



In [ ]:
# do not do this:
def bad_default(values: list[int] = []) -> int:
    values.append(0)
    return len(values)

# do this instead:
def good_default(values: list[int] = None) -> int:
    if values is None:
        values = []
        values.append(0)
    return len(values)


print(bad_default()) # 1
print(bad_default()) # 2
print(good_default()) # 1
print(good_default()) # 1



## Parallel processing
* It's possible to use parallel processing to speed up certain tasks.
* Example here is for using the high-level concurrent.futures pacakge for multithreading
```mermaid
    flowchart LR
        subgraph dependencies
            d0("data[0]")
            d1("data[1]")
        end
        subgraph parallel processing
            F0["function on thread 0"]
            F1["function on thread 1"]
        end
        d0 --> F0
        d1 --> F1

In [ ]:
import concurrent.futures

my_font = ImageFont.truetype("tmp/OpenSans-Regular.ttf", 72)

data = [
    (Path("tmp/base_image.jpg"), Path("tmp/test_2a.jpg"), Text(font=my_font, text="text multithreading a", alignment="center", color=(255, 0, 255))),
    (Path("tmp/base_image.jpg"), Path("tmp/test_2b.jpg"), Text(font=my_font, text="text multithreading b", alignment="center", color=(123, 123, 255)))
]

with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    futures_ = [executor.submit(add_text_2, src, dst, text, xy_at_center) for src, dst, text in data] # does not wait

## Asynchronus processing
* Using asyncio is also possible for specific use cases

In [ ]:
import asyncio
import aiofiles
import io

async def add_text_3(src: Path, dst: Path, text: Text, xy_function: Callable[[Image], (int, int)]) -> None:
    async with aiofiles.open(src, "rb") as f:
        stream = await f.read()
        img = Image.open(io.BytesIO(stream)).convert("RGB")
    text_w, text_h = xy_function(img)

    drawer = ImageDraw.Draw(img)
    drawer.multiline_text(xy=(text_w, text_h), text=text.text, font=text.font, align=text.alignment, fill=text.color)

    async with aiofiles.open(dst, "wb") as f:
        buffer = io.BytesIO()
        img.save(buffer, format="JPEG")
        await f.write(buffer.getbuffer())
    img.close()

my_font = ImageFont.truetype("tmp/OpenSans-Regular.ttf", 72)

data = [
    (Path("tmp/base_image.jpg"), Path("tmp/test_3a.jpg"), Text(font=my_font, text="async text a", alignment="center", color=(255, 0, 100))),
    (Path("tmp/base_image.jpg"), Path("tmp/test_3b.jpg"), Text(font=my_font, text="async test b", alignment="center", color=(100, 100, 255)))
]

async def runner():
    tasks = []
    async with asyncio.TaskGroup() as tg:
        for src, dst, text in data:
            tasks.append(tg.create_task(add_text_3(src, dst, text, xy_at_top_left)))

await runner()  # only run async functions like this in a notebook